In [48]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [49]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import warnings
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD, FastICA

import xgboost as xgb


import eli5
from eli5.sklearn import PermutationImportance
from eli5.permutation_importance import get_score_importances

In [50]:
import pandas as pd # 데이터 분석
import numpy as np # 행렬 연산, version: 1.6.1

import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgbm
import sklearn
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD, PCA, FastICA, FactorAnalysis, KernelPCA, DictionaryLearning
from sklearn.decomposition import IncrementalPCA, LatentDirichletAllocation,MiniBatchSparsePCA, SparsePCA

import itertools

In [51]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
sns.set(font_scale=2.5)

warnings.filterwarnings('ignore')

# 데이터 로드

In [52]:
data_dir = Path('../input/dankook')
sub_dir = Path('../output/')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'
sub_file = sub_dir / 'submission.csv'

target_column = 'class'
SEED = 2020

In [53]:
df_train = pd.read_csv(train_file, index_col=0)
df_test = pd.read_csv(test_file, index_col=0)

# 이상치 제거

In [54]:
# test의 MinMax 범위 넘는 행은 train에서 제거
train_shape = df_train.shape[0]

for col in df_train.columns[:18]:
    df_train = df_train.loc[np.logical_and(df_train[col]>=df_test[col].min(),
                            df_train[col]<=df_test[col].max())]

print('제거된 행 개수 :', train_shape - df_train.shape[0])

제거된 행 개수 : 77


In [55]:
df_train.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,19.8525,18.4318,17.7352,17.3037,17.0603,6.0481e-02,19.6298,18.2591,17.6158,17.2151,16.9944,6.3334,6.1396,1.1758,1.1765,1.1751,1.1754,1.1761,1.1165
std,1.9392,1.6598,1.4586,1.3141,1.3310,2.9110e-01,1.9114,1.6594,1.4672,1.3250,1.3232,8.8817,8.5522,0.1163,0.1181,0.1147,0.1155,0.1171,0.9234
min,2.2651,-12.4441,7.7314,7.7115,-9.2548,-2.5915e+01,-30.6337,-18.6560,-8.7560,-3.6973,0.2159,1.0000,1.0000,1.0001,1.0001,1.0002,1.0002,1.0002,0.0000
25%,18.7244,17.4750,16.8773,16.5240,16.2890,3.5724e-05,18.5639,17.3486,16.7874,16.4532,16.2343,1.0000,1.0000,1.0883,1.0885,1.0878,1.0881,1.0883,0.0000
50%,19.4195,18.1405,17.5259,17.1498,16.9177,4.7153e-02,19.2647,18.0225,17.4434,17.0874,16.8694,2.0000,2.0000,1.1794,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0728,18.4279,18.0074,17.7288,9.4606e-02,20.1976,18.8883,18.2908,17.9070,17.6555,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,49.1436,46.3383,45.1299,32.8634,52.6127,4.4620e+01,30.7779,30.6132,31.2940,30.5509,28.5710,44.0000,42.0000,2.0491,2.0786,2.0205,2.0347,2.0637,2.0000


In [56]:
df_test.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
count,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,19.8598,18.4371,17.7397,17.3065,17.1490,6.0083e-02,19.6350,18.2638,17.6193,17.2178,16.8702,6.3499,6.1578,1.1760,1.1767,1.1753,1.1756,1.1763
std,1.9543,1.7030,1.4786,1.3207,24.6431,3.4684e-01,1.9210,1.6657,1.4760,1.3324,35.4367,8.8728,8.5509,0.1164,0.1182,0.1148,0.1156,0.1173
min,-0.3385,-51.1753,-5.4387,5.3267,-39.5272,-3.0149e+01,-30.6337,-18.6560,-8.7560,-3.6973,-9999.0000,1.0000,1.0000,1.0001,1.0001,1.0002,1.0002,1.0001
25%,18.7273,17.4780,16.8820,16.5255,16.2882,3.3645e-05,18.5709,17.3522,16.7906,16.4562,16.2310,1.0000,1.0000,1.0881,1.0883,1.0877,1.0879,1.0882
50%,19.4261,18.1451,17.5299,17.1560,16.9212,4.7115e-02,19.2674,18.0247,17.4487,17.0912,16.8733,2.0000,2.0000,1.1794,1.1793,1.1794,1.1794,1.1793
75%,20.4344,19.0793,18.4345,18.0080,17.7333,9.4769e-02,20.1999,18.8948,18.2952,17.9109,17.6578,5.0000,5.0000,1.2278,1.2262,1.2294,1.2289,1.2269
max,56.8471,94.3591,46.6913,33.0259,6976.3922,4.6390e+01,30.8899,30.9529,31.6536,30.9478,28.6441,44.0000,42.0000,2.0502,2.0797,2.0216,2.0358,2.0648


# 변수 생성 

In [57]:
for i in ['u','g','r','i','z']:
    df_train[f'ugriz_{i}'] = df_train[i]
    df_test[f'ugriz_{i}'] = df_test[i]
df_train.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)
df_test.drop(columns=['u','g','r','i','z'], axis=1, inplace=True)

In [58]:
ugriz_col = [c for c in df_train.columns if c.find('ugriz') != -1]
dered_col = [c for c in df_train.columns if c.find('dered') != -1]
airmass_col = [c for c in df_train.columns if c.find('airmass') != -1]

In [59]:
# zip 함수를 이용하여 각 Row별, Magnitude별, max, min, max-min, std, sum을 구한다.
for prefix , g in zip(['ugriz','dered','airmass'], [ugriz_col, dered_col, airmass_col]):
    df_train[f'{prefix}_max'] = df_train[g].max(axis=1)
    df_test[f'{prefix}_max'] = df_test[g].max(axis=1)
    
    df_train[f'{prefix}_min'] = df_train[g].min(axis=1)
    df_test[f'{prefix}_min'] = df_test[g].min(axis=1)
    
    df_train[f'{prefix}_diff'] = df_train[f'{prefix}_max'] - df_train[f'{prefix}_min']
    df_test[f'{prefix}_diff'] = df_test[f'{prefix}_max'] - df_test[f'{prefix}_min']
    
    df_train[f'{prefix}_std'] = df_train[g].std(axis=1)
    df_test[f'{prefix}_std'] = df_test[g].std(axis=1)
    
    df_train[f'{prefix}_sum'] = df_train[g].sum(axis=1)
    df_test[f'{prefix}_sum'] = df_test[g].sum(axis=1)

In [60]:
# diff feature 추가 예: z - i
# itertools combinations을 활용하여 전체 magnitude에서 diff를 구함
# 총 105가지 조합이 나옴. 여기서 안 좋은 것은 permutation importance를 활용하여 제거할 예정
diff_feature = []
for c1, c2 in itertools.combinations(ugriz_col[::-1]+dered_col[::-1]
                                     +airmass_col[::-1],2):
    new_c = f'{c1}_{c2}_diff'
    df_train[new_c] = df_train[c1]-df_train[c2]
    df_test[new_c] = df_test[c1]-df_test[c2]
    diff_feature.append(new_c)
    

In [61]:
# 각 maginitude 별 max-max, min-min, sum-sum 을 구함

for c in itertools.combinations(['ugriz','dered','airmass'],2):
    df_train[f'{c[0]}_{c[1]}_max_diff'] = df_train[f'{c[0]}_max'] - df_train[f'{c[1]}_max']
    df_test[f'{c[0]}_{c[1]}_max_diff'] = df_test[f'{c[0]}_max'] - df_test[f'{c[1]}_max']
    
    df_train[f'{c[0]}_{c[1]}_min_diff'] = df_train[f'{c[0]}_min'] - df_train[f'{c[1]}_min']
    df_test[f'{c[0]}_{c[1]}_min_diff'] = df_test[f'{c[0]}_min'] - df_test[f'{c[1]}_min']
    
    df_train[f'{c[0]}_{c[1]}_sum_diff'] = df_train[f'{c[0]}_sum'] - df_train[f'{c[1]}_sum']
    df_test[f'{c[0]}_{c[1]}_sum_diff'] = df_test[f'{c[0]}_sum'] - df_test[f'{c[1]}_sum']

In [62]:
# 아래와 같은 공식들을 구현함
# http://classic.sdss.org/dr4/algorithms/sdssUBVRITransform.html 

def make_2flux_feature(train, test, c1, c2, func, mag_list=None):
    
    for c in mag_list:
        x=train[f'{c}_{c1}'].values
        y=train[f'{c}_{c2}'].values
        train[f'{c}_{func.__name__}'] = func(x,y)
        
        x=test[f'{c}_{c1}'].values
        y=test[f'{c}_{c2}'].values
        test[f'{c}_{func.__name__}'] = func(x,y)
        
def UB_jester(x1,x2):
    return 0.75*(x1-x2)-0.81

make_2flux_feature(df_train,df_test, 'u','g',UB_jester,['ugriz'])

def BV_jester(x1,x2):
    return 0.62*(x1-x2)+0.15

make_2flux_feature(df_train,df_test, 'g','r',BV_jester,['ugriz'])

def VR_jester(x1,x2):
    return 0.38*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i',VR_jester,['ugriz'])

def RcIc_jester(x1,x2):
    return 0.72*(x1-x2)+0.27

make_2flux_feature(df_train, df_test, 'r','i', RcIc_jester, ['ugriz'])

def B_jester(x1,x2):
    return x2+0.17*(x1-x2)+0.11

make_2flux_feature(df_train,df_test, 'u','g', B_jester, ['ugriz'])

def V_jester(x1,x2):
    return x1-0.52*(x1-x2)-0.03

make_2flux_feature(df_train,df_test, 'g','r',V_jester, ['ugriz'])

In [63]:
# https://www.sdss.org/dr16/algorithms/segue_target_selection/#Legacy

# 아래 나열된 모든 ugriz 등급은 달리 명시되지 않는 한 dereddened PSF mag입니다.
# dered_X 사용하면 될듯, 일단은 잘 몰라서 ugriz_X 버전과 dered_X 버전을 비교해서 사용하면 될듯.

# ugriz_X 버전

all_data = pd.concat([df_train,df_test], axis=0)
all_data.fillna(-1, inplace=True)

all_data['ugriz_icolor'] = all_data['ugriz_u']*(-0.436) + all_data['ugriz_g']*(1.129) + all_data['ugriz_r']*(-0.119) + all_data['ugriz_i']*(-0.574) + 0.1984

all_data['ugriz_ucolor'] = all_data['ugriz_u']*(-0.249) + all_data['ugriz_g']*(0.794) + all_data['ugriz_r']*(-0.555) + 0.234

all_data['ugriz_p1'] = (all_data['ugriz_u']-all_data['ugriz_g'])*(0.91) + (all_data['ugriz_g']-all_data['ugriz_r'])*(0.415)- 1.280

all_data['ugriz_r_std_div'] = all_data['ugriz_r']/all_data['ugriz_r'].std()

In [64]:
# https://www.sdss.org/dr16/algorithms/legacy_target_selection/
# 멸망과 관련 있는 거라고 함.
# dered_X 가 어떤 곳에서는 멸망과 관련 있는 데이터였음.
# ugriz_X를 붉어짐에 대한 걸로 사용하다면, 여기서는 dered_X를 멸망과 관련 있는 것과 사용하면 될듯

all_data['dered_orthogonal'] = (all_data['dered_r']-all_data['dered_i'])-(all_data['dered_g']-all_data['dered_r'])/4-0.18

all_data['dered_parallel'] = 0.7*(all_data['dered_g']-all_data['dered_r']) + 1.2*((all_data['dered_r']-all_data['dered_i'])-0.18)

In [65]:
# https://www.sdss.org/dr12/algorithms/magnitudes/
# 문서에는 psfMag에 대한 데이터인데, 그것을 나는 ugriz로 사용하고 있으니까 urgiz로 사용

color_list = ['u', 'g', 'r', 'i', 'z']
b_list = [1.4*10e-10, 0.9*10e-10, 1.2*10e-10, 1.8*10e-10, 7.4*10e-10]
f0_list = [24.63, 25.11, 24.80, 24.36, 22.83]
for c, b, f0 in zip(color_list, b_list, f0_list):
    all_data[f'ugriz_{c}_asinh'] = -2.5*np.log(10)*(np.arcsinh((all_data[f'ugriz_{c}']/f0)/(2*b))+np.log(b))

In [66]:
all_data['nObserve'] = all_data['nObserve'].apply(np.log1p)
all_data['d_obs_det'] = all_data['nObserve'] - all_data['nDetect']

# 데이터셋 분리

In [20]:
# train set
X = all_data.loc[all_data['class'] != -1 , :]
X.drop(columns='class',inplace=True,axis=1)
y = all_data.loc[all_data['class'] != -1, 'class']
y.astype(int)

# test set
test = all_data.loc[all_data['class'] == -1, :]
test.drop(columns='class', inplace=True,axis=1)

# train set split
SEED = 2020
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state = SEED)

In [21]:
df_train.shape, df_test.shape, X.shape, y.shape, test.shape

((319923, 154), (80000, 153), (319923, 165), (319923,), (80000, 165))

In [22]:
X.describe()

,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,ugriz_u,ugriz_g,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_min,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_g_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_r_diff,ugriz_i_ugriz_g_diff,ugriz_i_ugriz_u_diff,...,dered_r_airmass_g_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_i_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_r_diff,dered_u_airmass_g_diff,dered_u_airmass_u_diff,airmass_z_airmass_i_diff,airmass_z_airmass_r_diff,airmass_z_airmass_g_diff,airmass_z_airmass_u_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_min_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,ugriz_UB_jester,ugriz_BV_jester,ugriz_VR_jester,ugriz_RcIc_jester,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,ugriz_r_std_div,dered_orthogonal,dered_parallel,ugriz_u_asinh,ugriz_g_asinh,ugriz_r_asinh,ugriz_i_asinh,ugriz_z_asinh,d_obs_det
count,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,...,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,6.0481e-02,19.6298,18.2591,17.6158,17.2151,16.9944,1.4862,6.1396,1.1758,1.1765,1.1751,1.1754,1.1761,19.8525,18.4318,17.7352,17.3037,17.0603,19.8847,17.0145,2.8701,1.1633,90.3835,19.6647,16.9415,2.7232,1.1041,89.7141,1.1792,1.1725,6.7146e-03,2.6544e-03,5.8791,-0.2434,-0.6749,-1.3715,-2.7921,0.0659,-0.1547,-0.5554,-1.1988,-2.5694,15.8842,15.8849,15.8852,15.8838,15.8846,-0.4315,-1.1281,-2.5487,...,16.4392,16.4400,-1.3707,17.0830,17.0837,17.0840,17.0826,17.0833,18.4536,18.4543,18.4546,18.4532,18.4540,0.0007,0.0010,-0.0004,0.0004,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.2200,0.0731,0.6694,18.7055,15.8421,84.5044,18.4855,15.7690,83.8351,0.2555,0.5819,0.4340,0.5807,18.7833,18.0396,0.3094,0.0826,0.3019,12.1259,0.0599,0.7152,1.2671,1.8034,1.9489,1.9851,1.6964,-4.6534
std,2.9110e-01,1.9114,1.6594,1.4672,1.3250,1.3232,0.9061,8.5522,0.1163,0.1181,0.1147,0.1155,0.1171,1.9392,1.6598,1.4586,1.3141,1.3310,1.9642,1.2381,1.5576,0.6366,7.1184,1.9254,1.2544,1.4738,0.5997,7.1531,0.1183,0.1143,5.8399e-03,2.3081e-03,0.5815,0.5068,0.8029,1.1437,1.5900,0.2909,0.5392,0.8230,1.1530,1.5710,1.32

In [23]:
test.describe()

,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,ugriz_u,ugriz_g,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_min,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_g_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_r_diff,ugriz_i_ugriz_g_diff,ugriz_i_ugriz_u_diff,...,dered_r_airmass_g_diff,dered_r_airmass_u_diff,dered_g_dered_u_diff,dered_g_airmass_z_diff,dered_g_airmass_i_diff,dered_g_airmass_r_diff,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_r_diff,dered_u_airmass_g_diff,dered_u_airmass_u_diff,airmass_z_airmass_i_diff,airmass_z_airmass_r_diff,airmass_z_airmass_g_diff,airmass_z_airmass_u_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_min_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,ugriz_UB_jester,ugriz_BV_jester,ugriz_VR_jester,ugriz_RcIc_jester,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,ugriz_r_std_div,dered_orthogonal,dered_parallel,ugriz_u_asinh,ugriz_g_asinh,ugriz_r_asinh,ugriz_i_asinh,ugriz_z_asinh,d_obs_det
count,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,8.0000e+04,8.0000e+04,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,...,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000,80000.0000
mean,6.0083e-02,19.6350,18.2638,17.6193,17.2178,16.8702,1.4903,6.1578,1.1760,1.1767,1.1753,1.1756,1.1763,19.8598,18.4371,17.7397,17.3065,17.1490,19.9839,17.0133,2.9706,1.2082,90.4921,19.6729,16.8157,2.8572,1.1640,89.6061,1.1794,1.1727,6.7318e-03,2.6612e-03,5.8799,-0.1575,-0.5908,-1.2882,-2.7109,0.2787,-0.0689,-0.4704,-1.1148,-2.4860,15.9726,15.9733,15.9736,15.9722,15.9730,-0.4333,-1.1306,-2.5533,...,16.4426,16.4434,-1.3712,17.0875,17.0882,17.0885,17.0870,17.0878,18.4586,18.4593,18.4596,18.4582,18.4590,0.0007,0.0010,-0.0004,0.0004,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.3109,0.1975,0.8860,18.8045,15.8406,84.6122,18.4936,15.6431,83.7262,0.2571,0.5824,0.4346,0.5819,18.7890,18.0445,0.3100,0.0824,0.3041,12.1290,0.0604,0.7170,1.2678,1.8060,1.9531,1.9843,1.6981,-4.6675
std,3.4684e-01,1.9210,1.6657,1.4760,1.3324,35.4367,0.9055,8.5509,0.1164,0.1182,0.1148,0.1156,0.1173,1.9543,1.7030,1.4786,1.3207,24.6431,24.6764,1.2616,24.6651,11.0261,25.5608,1.9296,35.4342,35.4250,15.8383,36.2002,0.1185,0.1144,5.8600e-03,2.3160e-03,0.5822,24.6299,24.6382,24.6502,24.6773,60.0184,24.6307,24.6389,24.6522,24.6755,24.6420,24.6420,24.6420,24.6419,24.6420,0.5820,0.9922,1.4322,...,1.4704,1.4703,0.7

# feature 선택

In [24]:
# Xgboost
xgb_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'tree_method': 'auto',
    'objective': 'multi:softmax',
    'num_class': 3,
    'random_state': 2020
}

In [25]:
xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_clf.fit(X_train,y_train)
p = xgb_clf.predict(X_val)
print(accuracy_score(p,y_val))

0.9344113693905831


In [26]:
perm = PermutationImportance(xgb_clf, random_state=2020).fit(X_val,y_val)

In [27]:
eli5.show_weights(perm, top = 200, feature_names = X_val.columns.tolist())

Weight,Feature
0.4358 ± 0.0010,redshift
0.0637 ± 0.0012,dered_orthogonal
0.0167 ± 0.0013,dered_g_dered_u_diff
0.0036 ± 0.0002,dered_z_dered_i_diff
0.0035 ± 0.0007,ugriz_r_dered_u_diff
0.0030 ± 0.0003,ugriz_ucolor
0.0026 ± 0.0002,dered_r_dered_g_diff
0.0025 ± 0.0007,dered_r_dered_u_diff
0.0018 ± 0.0002,dered_i
0.0009 ± 0.0002,ugriz_icolor


In [28]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_val.columns.tolist())
top = feature_importances.loc[feature_importances[0]>0.0000]
best_feature = top.index.tolist()

print(top.shape, best_feature)

(111, 1) ['redshift', 'dered_g', 'dered_i', 'dered_z', 'nObserve', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'ugriz_r', 'ugriz_i', 'ugriz_z', 'ugriz_max', 'ugriz_diff', 'ugriz_std', 'ugriz_sum', 'dered_max', 'dered_diff', 'dered_sum', 'airmass_max', 'airmass_min', 'airmass_diff', 'airmass_std', 'airmass_sum', 'ugriz_z_ugriz_i_diff', 'ugriz_z_ugriz_r_diff', 'ugriz_z_ugriz_u_diff', 'ugriz_z_dered_z_diff', 'ugriz_z_dered_i_diff', 'ugriz_z_dered_r_diff', 'ugriz_z_dered_g_diff', 'ugriz_z_dered_u_diff', 'ugriz_z_airmass_z_diff', 'ugriz_z_airmass_i_diff', 'ugriz_z_airmass_r_diff', 'ugriz_z_airmass_u_diff', 'ugriz_i_ugriz_u_diff', 'ugriz_i_dered_z_diff', 'ugriz_i_dered_i_diff', 'ugriz_i_dered_g_diff', 'ugriz_i_dered_u_diff', 'ugriz_i_airmass_z_diff', 'ugriz_i_airmass_i_diff', 'ugriz_i_airmass_u_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_ugriz_u_diff', 'ugriz_r_dered_i_diff', 'ugriz_r_dered_g_diff', 'ugriz_r_dered_u_diff', 'ugriz_r_airmass_i_diff', 'ugriz_g_ugriz_u_diff', 'ugriz_g_dered_

In [29]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_val.columns.tolist())
top = feature_importances.loc[feature_importances[0]>0.0001]
best_feature = top.index.tolist()

print(top.shape, best_feature)

(67, 1) ['redshift', 'dered_g', 'dered_i', 'dered_z', 'nObserve', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'ugriz_r', 'ugriz_i', 'ugriz_z', 'ugriz_max', 'ugriz_diff', 'ugriz_std', 'ugriz_sum', 'dered_max', 'dered_sum', 'airmass_max', 'airmass_min', 'airmass_std', 'airmass_sum', 'ugriz_z_ugriz_i_diff', 'ugriz_z_ugriz_r_diff', 'ugriz_z_ugriz_u_diff', 'ugriz_z_dered_z_diff', 'ugriz_z_dered_i_diff', 'ugriz_z_dered_r_diff', 'ugriz_i_ugriz_u_diff', 'ugriz_i_dered_z_diff', 'ugriz_i_dered_g_diff', 'ugriz_i_dered_u_diff', 'ugriz_i_airmass_i_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_ugriz_u_diff', 'ugriz_r_dered_i_diff', 'ugriz_r_dered_u_diff', 'ugriz_r_airmass_i_diff', 'ugriz_g_ugriz_u_diff', 'ugriz_g_dered_r_diff', 'ugriz_g_dered_g_diff', 'ugriz_g_dered_u_diff', 'ugriz_u_dered_z_diff', 'ugriz_u_dered_r_diff', 'dered_z_dered_i_diff', 'dered_z_dered_g_diff', 'dered_z_airmass_g_diff', 'dered_i_dered_r_diff', 'dered_i_dered_g_diff', 'dered_i_dered_u_diff', 'dered_i_airmass_z_diff', 'dered

In [30]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_val.columns.tolist())
top = feature_importances.loc[feature_importances[0]>0.0002]
best_feature = top.index.tolist()

print(top.shape, best_feature)

(41, 1) ['redshift', 'dered_i', 'nObserve', 'airmass_u', 'airmass_g', 'airmass_i', 'ugriz_sum', 'dered_sum', 'airmass_min', 'airmass_sum', 'ugriz_z_ugriz_i_diff', 'ugriz_z_ugriz_u_diff', 'ugriz_z_dered_z_diff', 'ugriz_z_dered_i_diff', 'ugriz_z_dered_r_diff', 'ugriz_i_ugriz_u_diff', 'ugriz_i_dered_z_diff', 'ugriz_i_dered_g_diff', 'ugriz_i_dered_u_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_dered_i_diff', 'ugriz_r_dered_u_diff', 'ugriz_r_airmass_i_diff', 'ugriz_g_ugriz_u_diff', 'ugriz_g_dered_u_diff', 'dered_z_dered_i_diff', 'dered_z_dered_g_diff', 'dered_z_airmass_g_diff', 'dered_i_dered_r_diff', 'dered_i_dered_u_diff', 'dered_r_dered_g_diff', 'dered_r_dered_u_diff', 'dered_g_dered_u_diff', 'dered_g_airmass_z_diff', 'airmass_z_airmass_i_diff', 'ugriz_B_jester', 'ugriz_V_jester', 'ugriz_icolor', 'ugriz_ucolor', 'ugriz_p1', 'dered_orthogonal']


In [31]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_val.columns.tolist())
top = feature_importances.loc[feature_importances[0]>0.0003]
best_feature = top.index.tolist()

print(top.shape, best_feature)

(27, 1) ['redshift', 'dered_i', 'airmass_u', 'airmass_g', 'airmass_i', 'airmass_min', 'ugriz_z_ugriz_i_diff', 'ugriz_z_dered_i_diff', 'ugriz_z_dered_r_diff', 'ugriz_i_ugriz_u_diff', 'ugriz_i_dered_g_diff', 'ugriz_i_dered_u_diff', 'ugriz_r_ugriz_g_diff', 'ugriz_r_dered_u_diff', 'ugriz_g_dered_u_diff', 'dered_z_dered_i_diff', 'dered_z_dered_g_diff', 'dered_z_airmass_g_diff', 'dered_i_dered_r_diff', 'dered_r_dered_g_diff', 'dered_r_dered_u_diff', 'dered_g_dered_u_diff', 'airmass_z_airmass_i_diff', 'ugriz_V_jester', 'ugriz_icolor', 'ugriz_ucolor', 'dered_orthogonal']


In [32]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_val.columns.tolist())
top = feature_importances.loc[feature_importances[0]>0.0004]
best_feature = top.index.tolist()

print(top.shape, best_feature)

(20, 1) ['redshift', 'dered_i', 'airmass_u', 'airmass_i', 'airmass_min', 'ugriz_z_dered_r_diff', 'ugriz_i_dered_g_diff', 'ugriz_r_dered_u_diff', 'ugriz_g_dered_u_diff', 'dered_z_dered_i_diff', 'dered_z_dered_g_diff', 'dered_i_dered_r_diff', 'dered_r_dered_g_diff', 'dered_r_dered_u_diff', 'dered_g_dered_u_diff', 'airmass_z_airmass_i_diff', 'ugriz_V_jester', 'ugriz_icolor', 'ugriz_ucolor', 'dered_orthogonal']


# 차원 축소까지 적용

permutation importance 중요도가 0보다 큰 것들 가지고 표준편차 값 구하고
차원 축소 적용 해보기

In [67]:
feature_importances=pd.DataFrame(perm.feature_importances_, index=X_val.columns.tolist())
top = feature_importances.loc[feature_importances[0]>0.0001]
best_feature = top.index.tolist()

In [69]:
intersect_good_feature = list(set(diff_feature).intersection(set(best_feature)))
all_data['diff_feature_std'] = all_data[intersect_good_feature].std(axis=1)

In [70]:
df_train = all_data.loc[all_data['class'] != -1 , :]
df_test = all_data.loc[all_data['class'] == -1, :]
df_test.drop(columns='class', inplace=True,axis=1)

In [74]:
df_train.describe()

,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,ugriz_u,ugriz_g,ugriz_r,ugriz_i,ugriz_z,ugriz_max,ugriz_min,ugriz_diff,ugriz_std,ugriz_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,ugriz_z_ugriz_i_diff,ugriz_z_ugriz_r_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_i_diff,ugriz_z_dered_r_diff,ugriz_z_dered_g_diff,ugriz_z_dered_u_diff,ugriz_z_airmass_z_diff,ugriz_z_airmass_i_diff,ugriz_z_airmass_r_diff,ugriz_z_airmass_g_diff,ugriz_z_airmass_u_diff,ugriz_i_ugriz_r_diff,ugriz_i_ugriz_g_diff,...,dered_g_airmass_g_diff,dered_g_airmass_u_diff,dered_u_airmass_z_diff,dered_u_airmass_i_diff,dered_u_airmass_r_diff,dered_u_airmass_g_diff,dered_u_airmass_u_diff,airmass_z_airmass_i_diff,airmass_z_airmass_r_diff,airmass_z_airmass_g_diff,airmass_z_airmass_u_diff,airmass_i_airmass_r_diff,airmass_i_airmass_g_diff,airmass_i_airmass_u_diff,airmass_r_airmass_g_diff,airmass_r_airmass_u_diff,airmass_g_airmass_u_diff,ugriz_dered_max_diff,ugriz_dered_min_diff,ugriz_dered_sum_diff,ugriz_airmass_max_diff,ugriz_airmass_min_diff,ugriz_airmass_sum_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,ugriz_UB_jester,ugriz_BV_jester,ugriz_VR_jester,ugriz_RcIc_jester,ugriz_B_jester,ugriz_V_jester,ugriz_icolor,ugriz_ucolor,ugriz_p1,ugriz_r_std_div,dered_orthogonal,dered_parallel,ugriz_u_asinh,ugriz_g_asinh,ugriz_r_asinh,ugriz_i_asinh,ugriz_z_asinh,d_obs_det,diff_feature_std,tsvd5_0,tsvd5_1,tsvd5_2,tsvd5_3,tsvd5_4
count,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,3.1992e+05,3.1992e+05,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,...,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000,319923.0000
mean,6.0481e-02,19.6298,18.2591,17.6158,17.2151,16.9944,1.4862,6.1396,1.1758,1.1765,1.1751,1.1754,1.1761,1.1165,19.8525,18.4318,17.7352,17.3037,17.0603,19.8847,17.0145,2.8701,1.1633,90.3835,19.6647,16.9415,2.7232,1.1041,89.7141,1.1792,1.1725,6.7146e-03,2.6544e-03,5.8791,-0.2434,-0.6749,-1.3715,-2.7921,0.0659,-0.1547,-0.5554,-1.1988,-2.5694,15.8842,15.8849,15.8852,15.8838,15.8846,-0.4315,-1.1281,...,17.0826,17.0833,18.4536,18.4543,18.4546,18.4532,18.4540,0.0007,0.0010,-0.0004,0.0004,0.0003,-0.0011,-0.0003,-0.0014,-0.0006,0.0008,0.2200,0.0731,0.6694,18.7055,15.8421,84.5044,18.4855,15.7690,83.8351,0.2555,0.5819,0.4340,0.5807,18.7833,18.0396,0.3094,0.0826,0.3019,12.1259,0.0599,0.7152,1.2671,1.8034,1.9489,1.9851,1.6964,-4.6534,6.1946,57.0962,-0.0354,0.0136,-0.0022,0.0043
std,2.9110e-01,1.9114,1.6594,1.4672,1.3250,1.3232,0.9061,8.5522,0.1163,0.1181,0.1147,0.1155,0.1171,0.9234,1.9392,1.6598,1.4586,1.3141,1.3310,1.9642,1.2381,1.5576,0.6366,7.1184,1.9254,1.2544,1.4738,0.5997,7.1531,0.1183,0.1143,5.8399e-03,2.3081e-03,0.5815,0.5068,0.8029,1.1437,1.5900,0.2909,0.5392,0.8230,1.1530,1.5710,1.3270,1.3270,1.3270,1.3270,1.3270,0.5494,0.9157,...,1.6537,1.6536,1.9046,1.9046,1.9046,1.9046,

In [73]:
def get_decomposition_feature(train, test, feature, param, decompose_func, prefix):
    n_components = param['n_components']
    de = decompose_func(**param)
    de_train = de.fit_transform(train[feature])
    de_test = de.transform(test[feature])
    train = pd.concat([train, pd.DataFrame(de_train, columns=[f'{prefix}_{c}' for c in range(n_components)])], axis=1)
    test = pd.concat([test, pd.DataFrame(de_test, columns=[f'{prefix}_{c}' for c in range(n_components)])], axis=1)
    return train, test

org_feature = ugriz_col+dered_col+airmass_col
# decompostion 해서 다시 feature로 추가, 원래 orignal feature만 사용하고 5개로 축소
decom_common_param = {'n_components': 5, 'random_state': 2020}
df_train, df_test = get_decomposition_feature(df_train, df_test, org_feature, decom_common_param, TruncatedSVD, 'tsvd5')
df_train, df_test = get_decomposition_feature(df_train, df_test, org_feature, decom_common_param, FastICA, 'ica5')

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
df_train